Query check

In [1]:
from core_pro import DataPipeLine


query = """
    select grass_date
    , count(distinct user_id) total_a1
    from mp_user.dws_user_login_1d__vn_s0_live
    where grass_date >= date'2022-01-01'
    group by 1
"""
print(DataPipeLine(query).query)

In [2]:
df = DataPipeLine(query).run_presto_to_df()

# 2. Threadpool query data

How to set multi-thread download with different dates

In [3]:
import polars as pl
from datetime import date, timedelta
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor



def download(date_input):
    """wrapper your download code into a function"""
    # setup file name
    path = Path.home() / 'VSCodeProjects/how-to-use/'
    file_name = path / f'{date_input}.parquet'

    # query
    query = f"""
    select grass_date
    , count(distinct user_id) total_a1
    from mp_user.dws_user_login_1d__vn_s0_live
    where grass_date = date'{date_input}'
    group by 1
    """

    # check if file exist (don't have to download file again)
    if not file_name.exists():
        df = DataPipeLine(query).run_presto_to_df()
        df.write_parquet(file_name, use_pyarrow=True)

In [4]:
yesterday = date.today() - timedelta(days=1)
dates = pl.date_range(start=date(2023, 9, 8), end=yesterday, interval='2d', eager=True)
run = list(dates)
run

In [5]:
max_workers = 4   # depend on your CPU
with ThreadPoolExecutor(max_workers) as executor:
    executor.map(download, run)